<a href="https://colab.research.google.com/github/harita-gr/ICTACK-AI-ML/blob/main/Assignment05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation Systems

The Jokes Dataset is given.
Build a collaborative filtering based recommendation system on jokes rating

In [18]:
import pandas as pd

In [19]:
data = pd.read_csv('/content/jokes-data.csv')

## 1. Data Analysis

In [20]:
data.head()

,id,user_id,joke_id,Rating
0,31030_110,31030,110,2.750
1,16144_109,16144,109,5.094
2,23098_6,23098,6,-6.438
3,14273_86,14273,86,4.406
4,18419_134,18419,134,9.375


In [21]:
data.shape

(1092059, 4)

In [22]:
data['user_id'].nunique()

40863

In [23]:
data['joke_id'].nunique()

139

In [24]:
data['Rating'].nunique()

641

In [25]:
data.isna().sum()

id         0
user_id    0
joke_id    0
Rating     0
dtype: int64

No missing values present in data.No further pre-processing required for the data set.

## 2. Load and Prepare the Data

In [26]:
!pip install scikit-surprise

In [27]:
from surprise import Dataset, Reader, KNNWithMeans, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [28]:
#Creating a sample of data set of 10000 since the dataset is too large to handle
subset_data = data.iloc[:10000, :]

# Load the data
reader = Reader(rating_scale=(-10, 10))  # Define the rating scale
df = Dataset.load_from_df(subset_data[['user_id', 'joke_id', 'Rating']], reader)

## 3. Use User-Based Collaborative Filtering

Here we are using 2 alogorithms


*   k-NN Means
*   SVD

We will perform cross validation as well.




In [30]:
sim_options = {
    'name': 'pearson',  # Use pearson similarity
    'user_based': True  # User-based CF
}

In [31]:
#Define models
svd_model = SVD()
knn_model = KNNWithMeans(sim_options=sim_options)

Cross Validation using Surprise

In [32]:
from surprise.model_selection import cross_validate

In [33]:
cross_validate(knn_model, df, measures=['RMSE'],cv=5,verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    5.2954  5.3260  5.4353  5.3850  5.2935  5.3470  0.0552  
Fit time          3.75    4.06    6.70    3.36    3.47    4.27    1.24    
Test time         0.05    0.12    0.14    0.05    0.09    0.09    0.04    


{'test_rmse': array([5.2953731 , 5.32601059, 5.43527482, 5.38504203, 5.29349226]),
 'fit_time': (3.747931718826294,
  4.055297613143921,
  6.699697017669678,
  3.3588268756866455,
  3.474003553390503),
 'test_time': (0.04943490028381348,
  0.12046599388122559,
  0.1437976360321045,
  0.04644012451171875,
  0.0886678695678711)}

In [34]:
cross_validate(svd_model, df, measures=['RMSE'],cv=5,verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    4.9910  4.9893  4.9129  4.9663  5.0784  4.9876  0.0535  
Fit time          0.26    0.24    0.26    0.42    0.36    0.31    0.07    
Test time         0.02    0.02    0.02    0.02    0.03    0.02    0.01    


{'test_rmse': array([4.99102662, 4.98929573, 4.91293115, 4.96630037, 5.07843487]),
 'fit_time': (0.2554302215576172,
  0.23569440841674805,
  0.2619948387145996,
  0.41829729080200195,
  0.36308836936950684),
 'test_time': (0.015743494033813477,
  0.015516042709350586,
  0.017864704132080078,
  0.016698598861694336,
  0.03120899200439453)}

SVD has a lower mean RMSE (4.9876), suggesting better predictive performance compared to KNNWithMeans (5.3470).

## 4. Making Prediction

In [37]:
userid = 18419.0
jokeid = 134.0

In [38]:
prediction = svd_model.predict(userid, jokeid)
prediction.est

8.676545559457791